<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
# os.chdir(f'./Transformer-Explainability')

# !pip install -r requirements.txt
# !pip install captum

In [ ]:
# !pip install captum==0.6.0
# !pip install matplotlib==3.3.2

In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

In [2]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


#Positive sentiment example

In [3]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

/home/gkoren2/work/git/clones/transformers/src/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[('[CLS]', 0.0), ('this', 0.4272907078266144), ('movie', 0.30951225757598877), ('was', 0.268684446811676), ('the', 0.339260071516037), ('best', 0.6288697123527527), ('movie', 0.28607645630836487), ('i', 0.1867041438817978), ('have', 0.10146278887987137), ('ever', 0.14219637215137482), ('seen', 0.19010096788406372), ('!', 0.5949175953865051), ('some', 0.0040426114574074745), ('scenes', 0.0334150567650795), ('were', 0.01868431642651558), ('ridiculous', 0.018948297947645187), (',', 0.0), ('but', 0.42987316846847534), ('acting', 0.43847790360450745), ('was', 0.5003233551979065), ('great', 1.0), ('.', 0.014387628063559532), ('[SEP]', 0.08716049790382385)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


#Negative sentiment example

In [4]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                1,
                                1,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.19125820696353912), ('really', -0.18891339004039764), ('didn', -0.28977063298225403), ("'", -0.006524990778416395), ('t', -0.36837172508239746), ('like', -0.15253733098506927), ('this', -0.18936631083488464), ('movie', -0.04035608842968941), ('.', -0.019366510212421417), ('some', -0.022951893508434296), ('of', -0.0), ('the', -0.02299492433667183), ('actors', -0.09489651769399643), ('were', -0.01353638619184494), ('good', -0.0323217548429966), (',', -0.00414715800434351), ('but', -0.0587456151843071), ('overall', -0.3360520005226135), ('the', -0.21848560869693756), ('movie', -0.054846927523612976), ('was', -0.6255497336387634), ('boring', -1.0), ('.', -0.030879611149430275), ('[SEP]', -0.052479859441518784)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


# Choosing class for visualization example

In [5]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 0
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.1979038566350937), ('hate', -1.0), ('that', -0.40287482738494873), ('i', -0.12505769729614258), ('love', -0.13071633875370026), ('you', -0.054671306163072586), ('.', -6.109004061727319e-06), ('[SEP]', -0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


In [6]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 1
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('i', 0.2725588381290436), ('hate', 0.17270229756832123), ('that', 0.2321109175682068), ('i', 0.17642885446548462), ('love', 1.0), ('you', 0.24655385315418243), ('.', 0.0), ('[SEP]', 0.0001573420304339379)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]
